# Udacity SDC Nanodegree - Advanced Lane Lines Project

## Some thoughts
I have spent much more time on this project than any previous ones. It has been a good practice for me to get familiar with many computer vision concepts and techniques, such as camera calibration and perspective transform. But to be frank, I am still not convinced that the algorithms used here are general and robust enough to be useful in practical self-driving, with the main reasons being:

1. There are too many "ad-hoc" assumptions on the image, e.g., the brightness/color of lanes compared to other objects, the standard of roads (e,g., for meter-per-pixel estimate), and etc. As a consequence, there are a lot of possibilities to break these assumptions, e.g., driving from urban roads to high way in another country.
2. The rules that encode the "lane-detection" knowledge are very complicated compared to other data-driven approach such as semantic segmentation. Usually this means the model has a bigger chance of being overfitted, e.g., when some parts of lanes are hidden. And this cannot be remedied by using more and more data! This makes the results of the method less reliable in practice.
3. The outcome of the method, namely curvature and position are usually based on rough estimatio, and may not be accurate enough for a steering purpose.

A lot of literature on lane detection by traditional computer vision can be found in previous publications, but I think most of the focuses have been shifted toward more data-driven approaches recently. I hope there will be more advanced techniques covered in the later part of the course to address those shortcomings.

## Code Organization

- The code is wraped in package `sdclane`, with main functions implemented in the following files:
    - `camera.py`: camera calibration
    - `line_detection.py`: line detection by several methods such as `sobel` in different color spaces
    - `transform.py`: perspective transform to get bird-eye view of lanes
    - `lane_detection.py`: main class `LaneDetector` builds pipeline from previous steps to detect/estimate lanes in images and videos.
    - `config.py`: configuration such as images for camera calibration and testing
    - `utility.py`: helper functions for pipeline and image io

In [1]:
%matplotlib inline

from sdclane import config, utility, camera, line_detection, lane_detection

import cv2
import matplotlib.pyplot as plt
import numpy as np

from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [2]:
# build lane detector
lane_detector = lane_detection.LaneDetector()

## Project Rubric Go-through

### 1. Camera Calibration
OpenCV functions or other methods were used to calculate the correct camera matrix and distortion coefficients using the calibration chessboard images provided in the repository. The distortion matrix should be used to un-distort the test calibration image provided as a demonstration that the calibration is correct

Image undistortion is implemented in `sdclane.camera.CameraCalibrator` class. 
- its `fit` method uses a set of chessboard_images and estimates distortion matrix and coefficients as `self.M` and `self.d`
- its `undistor` method takes a raw image and undistorts it accordingly
- during the implementation, I found some images provided in `camera_cal` folder have different specifications e.g., # of row/col corners. Those are simply ignored.
The results of undistortion will be shown below.

### 2. Pipeline for single images

#### 2.1 Distortion Correction
Distortion correction that was calculated via camera calibration has been correctly applied to each image.

#### 2.2 Line detection
At least two methods (i.e., color transforms, gradients) have been combined to create a binary image containing likely lane pixels. There is no "ground truth" here, just visual verification that the pixels identified as part of the lane lines are, in fact, part of the lines.

#### 2.3 Perspective transform
OpenCV function or other method has been used to correctly rectify each image to a "birds-eye view"

#### 2.4 Lane fitting
Methods have been used to identify lane line pixels in the rectified binary image. The left and right line have been identified and fit with a curved functional form (e.g., spine or polynomial).

#### 2.5 Lane parameter estimation
Here the idea is to take the measurements of where the lane lines are and estimate how much the road is curving and where the vehicle is located with respect to the center of the lane. The radius of curvature may be given in meters assuming the curve of the road follows a circle and the position of the vehicle within the lane may be given as meters off of center.

#### 2.6 visual validation
The fit from the rectified image has been warped back onto the original image and plotted to identify the lane boundaries. This should demonstrate that the lane boundaries were correctly identified.

### 3. Pipeline for video

#### 3.1 Lane detection
The image processing pipeline that was established to find the lane lines in images successfully processes the video. The output here should be a new video where the lanes are identified in every frame, and outputs are generated regarding the radius of curvature of the lane and vehicle position within the lane. The identification and estimation don't need to be perfect, but they should not be wildly off in any case. The pipeline should correctly map out curved lines and not fail when shadows or pavement color changes are present.

#### 3.2 Lane search for first frame
In the first few frames of video, the algorithm should perform a search without prior assumptions about where the lines are (i.e., no hard coded values to start with). Once a high-confidence detection is achieved, that positional knowledge may be used in future iterations as a starting point to find the lines.

#### 3.3 Tracking of different frames
As soon as a high confidence detection of the lane lines has been achieved, that information should be propagated to the detection step for the next frame of the video, both as a means of saving time on detection and in order to reject outliers (anomalous detections).

The algorithm works only partially with challenge video when certain assumption holds.

### 4. Conclusion
The Readme file submitted with this project includes a detailed description of what steps were taken to achieve the result, what techniques were used to arrive at a successful result, what could be improved about their algorithm/pipeline, and what hypothetical cases would cause their pipeline to fail.